# Sensor Projection

In this tutorial, we will demonstrate how to define a **sensor geometry** and project its trace onto the **Earth**.

## Setup

Let's import the necessary dependencies:

In [ ]:
import numpy as np
import pandas as pd

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected = True)

import ostk.mathematics as mathematics
import ostk.physics as physics

Then, we define some useful shortcuts:

In [ ]:
Object2 = mathematics.geometry.d2.Object
Point2 = mathematics.geometry.d2.objects.Point
Polygon2 = mathematics.geometry.d2.objects.Polygon
Point3 = mathematics.geometry.d3.objects.Point
Polygon3 = mathematics.geometry.d3.objects.Polygon
Ellipsoid = mathematics.geometry.d3.objects.Ellipsoid
Pyramid = mathematics.geometry.d3.objects.Pyramid

Length = physics.units.Length
Angle = physics.units.Angle
Scale = physics.time.Scale
Instant = physics.time.Instant
Duration = physics.time.Duration
Interval = physics.time.Interval
DateTime = physics.time.DateTime
LLA = physics.coordinate.spherical.LLA
Position = physics.coordinate.Position
Frame = physics.coordinate.Frame
Environment = physics.Environment
Geometry = physics.environment.object.Geometry
Earth = physics.environment.objects.celestial_bodies.Earth

---

## Scene

We first set up a simple scene, with the Earth only (the default `Environment` will suffice here):

In [ ]:
environment = Environment.default() ;

Then, we access the `Earth` object (managed by the `Environment`):

In [ ]:
earth = environment.access_object_with_name("Earth")

Once the `Earth` has been obtained, we can also get its geometry (an `Ellipsoid`, defined in `ITRF` in this case):

In [ ]:
earth_geometry = earth.get_geometry_in(Frame.ITRF())

## Sensor

Let's define a pyramidal geometry:

In [ ]:
apex = Point3(7000e3, 0.0, 0.0)
base = Polygon3(Polygon2([Point2(-1.0, -1.0), Point2(+1.0, -1.0), Point2(+1.0, +1.0), Point2(-1.0, +1.0)]), apex - np.array((0.8, 0.0, 0.0)), np.array((0.0, 1.0, 0.0)), np.array((0.0, 0.0, 1.0)))

pyramid = Pyramid(base, apex)

That we express in `ITRF`:

In [ ]:
sensor_geometry = Geometry(pyramid, Frame.ITRF())

## Intersection

Now that we have both Earth and sensor geometries clearly defined, we can compute their intersection:

In [ ]:
intersection_ITRF = sensor_geometry.intersection_with(earth_geometry)

And convert this 3D intersection into a 2D set of geodetic points:

In [ ]:
intersection_points = [Point2(lla.get_longitude().in_degrees(), lla.get_latitude().in_degrees()) for lla in [LLA.cartesian(point_ITRF.as_vector(), Earth.equatorial_radius, Earth.flattening) for point_ITRF in intersection_ITRF.access_composite().access_object_at(0).as_line_string()]]

We further convert this set into a Pandas Dataframe, as a very convenient way for storing / managing data in Python:

In [ ]:
intersection_df = pd.DataFrame([[float(intersection_point.x()), float(intersection_point.y())] for intersection_point in intersection_points], columns=['Longitude', 'Latitude']) ;

In [ ]:
intersection_df.head()

Now, we're ready to visualize the intersection on a map!

In [ ]:
data = []

data.append(
    dict(
        type = 'scattergeo',
        lon = intersection_df['Longitude'],
        lat = intersection_df['Latitude'],
        mode = 'lines',
        line = dict(
            width = 1,
            color = 'red',
        )
    )
)
    
layout = dict(
        title = None,
        showlegend = False, 
        geo = dict(
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )
    
fig = dict(data=data, layout=layout)
iplot(fig)

It is also possible to obtain the `WKT` representation of the intersection polygon:

In [ ]:
Polygon2(intersection_points).to_string(Object2.Format.WKT)

---